In [1]:
import cpmpy as cp

In [26]:
n = 9 # number of blocks
k = 3 # number of piles

start = [2, 0, 9, 6, 3, 1, 5, 4, 0]
goal =  [3, 8, 4, 5, 0, 9, 0, 0, 1]

# n = 5
# k = 3
# start = [5, 0, 2, 0, 4]
# goal = [2, 3, 0, 5, 0]


In [ ]:

import cpmpy as cp
from cpmpy.tools.explain import mus

FROM, TO = 0,1
nCubes, horizon = n + 1, n * k + 1  # nCubes includes the dummy cube 0

state = cp.intvar(0,n,   shape=(nCubes, horizon), name="state")
count = cp.intvar(0,k,   shape=(nCubes, horizon), name="count")
done = cp.boolvar(       shape=horizon, name="done")
move = cp.intvar(0,n+1,  shape=(horizon,2), name="move")
locked = cp.boolvar(     shape=(nCubes, horizon), name="locked")

model = cp.Model()
model.minimize(horizon - cp.sum(done))

# dummy block is always on top

model += state[0,:] == 0

# define start state
model += state[1:, 0] == start

# end state (redundant constraint)
model += state[1:,-1] == goal

# define doneness
for t in range(horizon):
    model += cp.all(state[1:, t] == goal) == done[t]

# ensure we have two phases
model += cp.Increasing(done)

# define moves
for t in range(horizon):
    model += move[t,TO] == state[move[t, FROM],t]

       
for t in range(1, horizon):
    is_done = done[t-1]
    
    model += cp.IfThenElse(done[t-1],
                           cp.all(state[1:, t-1] == state[1:, t]), # don't move
                           # if a block is changed, it has moved
                           cp.all([(state[b,t-1] != state[b,t]) == (b == move[t,FROM]) for b in range(nCubes)]) & 
                           # if a block is moved, it should have been free
                           (count[move[t, FROM], t-1] == 0)
    )

    # no more moves once finished
    model += is_done.implies(move[t,FROM] == 0)

# count nb of times a block occurs
model += count[0, :] >= 1
model += count[1:, :] <= 1

for t in range(horizon):
    model += cp.GlobalCardinalityCount(state[1:, t], list(range(0,nCubes)), count[:, t], closed=True)

    
# redundant constraints, speeding up search
# prevent do-undo moves
for t in range(1,horizon-1):

    model += (~done[t]).implies(cp.all([move[t, FROM] != move[t+1, FROM],
                                        move[t, TO]   != move[t+1, FROM],
                                        move[t, TO]   != move[t+1, TO],
                                        move[t, FROM] != move[t, TO]]))

# some cubes are locked in end position
for t in range(horizon):
    for b in range(1,n):
        model += locked[b,t] == ((state[b,t] == goal[b]) & (locked[goal[b]][t] if goal[b] != 0 else True))
        
# don't move locked blocks
for t in range(horizon-1):
    for b in range(1,n):
        model += locked[b,t].implies(state[b,t+1] == goal[b])
        model += locked[b,t].implies(move[t+1, FROM] != b)


######################################
if model.solve(solver="choco") is False:
    print("UNSAT")
    print("Finding MUS")
    for c in mus(model.constraints):
        print("-", c)

else:
    print(model.status())
    obj = model.objective_value()
    print(model.objective_value())

In [25]:
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

def get_piles(state):
    state = [x - 1 for x in state]
    towers = []
    for next_idx in range(len(state)):
        if next_idx in state: continue
        tower = [next_idx+1]
        while next_idx >= 0:
            x = state[next_idx]
            tower.append(int(x+1))
            next_idx = x
        towers.append(tower[:-1])
    return sorted(towers, key=lambda x :x[0])

def get_towers(start, moves):
    

    towers = [get_piles(start)]
    print(towers)
    for f,t in moves:
        new_state = deepcopy(towers[-1])
        if f == 0: 
            pass
        elif t == 0: # move to new tower
            from_tower = max(new_state, key=lambda p : p[0] == f if len(p) else False)
            from_tower.pop(0)
            new_state.append([f])
        else:
            from_tower = max(new_state, key=lambda p : p[0] == f if len(p) else False)
            to_tower = max(new_state, key=lambda p : p[0] == t if len(p) else False)
            assert from_tower[0] == f, f"Unable to execute move {(f,t)} with towers {new_state}"
            assert to_tower[0] == t, f"Unable to execute move {(f,t)} with towers {new_state}"
            to_tower.insert(0, from_tower.pop(0))
                
        towers.append(new_state)
    
    return towers
        
towers = get_towers(start, move.value())

colors = [
    '#1f77b4',  # blue
    '#ff7f0e',  # orange
    '#2ca02c',  # green
    '#d62728',  # red
    '#9467bd',  # purple
    '#8c564b',  # brown
    '#e377c2',  # pink
    '#7f7f7f',  # gray
    '#bcbd22',  # yellow
    '#17becf'   # cyan
]


def draw_piles(piles):
    
    for p_idx, pile in enumerate(piles):
        for b_idx, block in enumerate(reversed(pile)):
            x0, y0 = p_idx-0.4 , b_idx
            x1, y1 = x0+0.8, y0+1
            yield go.Scatter(
                    x=[x0, x1, x1, x0, x0],
                    y=[y0, y0, y1, y1, y0],
                    mode='lines',
                    line=dict(color="black"),
                    fill='toself',
                    fillcolor=colors[block]
                    )
            yield go.Scatter(
                    x=[x0 +0.4], y=[y0+0.4],
                    mode='text',
                    text=str(block)
                    )

fig = go.Figure(
    data=list(draw_state(state.value()[1:, 0])),
    layout=go.Layout(
        xaxis=dict(range=[-0.5, 3], autorange=False),
        yaxis=dict(range=[0, 5], autorange=False),
        title=dict(text="Blocks world"),
        updatemenus=[dict(type="buttons", 
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, {"frame": {"duration": 1000, "redraw": False}}])])]
    ),
    frames=[go.Frame(data=list(draw_piles(piles))) for piles in towers]
)

fig.show()
# draw_state([2, 0, 9, 6, 3, 1, 5, 4, 0])


[[[1, 5, 4], [3, 2]]]


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Function to convert the state into piles
def get_piles_from_state(state):
    k = len(state)
    # Create a list of lists for the piles
    piles = [[] for _ in range(k)]
    is_on_top = [True] * k  # To track which blocks are on top
    
    # Iterate over the state and assign blocks to piles
    for i, s in enumerate(state):
        if s == 0:  # Block i is on top of a pile
            piles[i].append(i + 1)
        else:
            piles[s - 1].append(i + 1)  # Place block i on block (s-1)
            is_on_top[s - 1] = False  # Mark block (s-1) as not on top
    
    # Ensure the piles are correctly ordered (top-down)
    for i in range(k):
        if is_on_top[i] and not piles[i]:  # Empty piles that have no block
            piles[i] = [i + 1]
    
    return [pile for pile in piles if pile]  # Remove empty piles

# Function to draw a frame from a given state
def draw_frame(state, width=1.0, height=1.0, gap=0.1):
    piles = get_piles_from_state(state)
    num_piles = len(piles)
    
    # Determine the layout of the plot
    fig = go.Figure()
    for pile_index, pile in enumerate(piles):
        for block_index, block in enumerate(reversed(pile)):
            # Calculate the position of each block
            x0 = pile_index * (width + gap)
            y0 = block_index * (height + gap)
            
            # Draw the block as a rectangle with the block number
            fig.add_shape(type='rect',
                          x0=x0, y0=y0,
                          x1=x0 + width, y1=y0 + height,
                          line=dict(color='black'),
                          fillcolor='skyblue')
            
            # Add the block number as text in the middle of the rectangle
            fig.add_trace(go.Scatter(
                x=[x0 + width / 2],
                y=[y0 + height / 2],
                text=str(block),
                mode='text',
                textfont=dict(size=14)
            ))

    # Layout settings
    fig.update_layout(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        width=800,
        height=400,
        showlegend=False,
        margin=dict(l=10, r=10, t=10, b=10)
    )

    return fig

# Sample states for animation (sequence of states)
states = [
    [0, 0, 0, 0],  # Initial configuration
    [2, 0, 0, 0],  # Block 1 moves on Block 2
    [2, 3, 0, 0],  # Block 2 moves on Block 3
    [0, 3, 1, 0]   # Block 3 moves on Block 4
]

# Create the figure
fig = go.Figure(
    data=
    layout=go.Layout(
        title="Blocks World Animation",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, dict(frame=dict(duration=1, redraw=True), fromcurrent=True)])]
        )]
    ),
    frames=[go.Frame(data=draw_frame(state).data) for state in states]
)

# Show the animation
fig.show()
